In [0]:
!pip install metpy

In [0]:
import numpy as np
import pandas as pd
from datetime import datetime
import metpy
import metpy.io

In [0]:
###Download the METAR data
!wget ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2000/64010KMSN200012.dat

In [0]:
###Read the METAR file. An error will result after the last line, but the data should be saved to the decoded_metar DataFrame.
### OLD. Not flexible to skipping lines
#month = 7
#with open('64010KMSN200007.dat','r') as metar_file:
#    metar_line = metar_file.readline()
#    metar_line_clean = metar_line[53:] #Get rid of first 53 characters in each line, superfluous info
#    decoded_metar = metpy.io.parse_metar_to_dataframe(metar_line_clean, year=2000, month=month) #decode the METAR
#    decoded_metar = decoded_metar.set_index(keys='date_time')
#    cnt = 1 #Start a counter
#    while metar_line:
#        metar_line = metar_file.readline()
#        metar_line_clean = metar_line[53:]
#        decoded_metar_line = metpy.io.parse_metar_to_dataframe(metar_line_clean, year=2000, month=month)
#        decoded_metar_line = decoded_metar_line.set_index(keys='date_time')
#        decoded_metar = decoded_metar.append(decoded_metar_line)
#        decoded_metar
#        cnt = cnt+1

In [0]:
###Read the METAR file. An error will result after the last line, but the data should be saved to the decoded_metar DataFrame.
###NEW. Will skip any lines that the parser doesn't like.
month = 12
with open('64010KMSN200012.dat','r') as metar_file:
    metar_line = metar_file.readline()
    metar_line_clean = metar_line[53:] #Get rid of first 53 characters in each line, superfluous info
    decoded_metar = metpy.io.parse_metar_to_dataframe(metar_line_clean, year=2000, month=month) #decode the METAR
    decoded_metar = decoded_metar.set_index(keys='date_time')
    cnt = 1 #Start a counter
    while metar_line:
        try:
            metar_line = metar_file.readline()
            metar_line_clean = metar_line[53:]
            decoded_metar_line = metpy.io.parse_metar_to_dataframe(metar_line_clean, year=2000, month=month)
            decoded_metar_line = decoded_metar_line.set_index(keys='date_time')
            decoded_metar = decoded_metar.append(decoded_metar_line)
            decoded_metar
            cnt = cnt+1
        except:
            print("Skipping this entry...")

In [0]:
decoded_metar
#decoded_metar.tail(14)

In [0]:
##Standard adjustment
df2 = decoded_metar.iloc[[-72]]
number_of_lines_to_fix = np.arange(71)
for i in number_of_lines_to_fix:
    i = 71-i
    df2 = df2.append(decoded_metar.iloc[[-i]])
df2

##Manual adjustment
#df2 = decoded_metar.iloc[[-14]]
#number_of_lines_to_fix = np.arange(13)
#for i in number_of_lines_to_fix:
#    i = 13-i
#    df2 = df2.append(decoded_metar.iloc[[-i]])
#df2

In [0]:
df2.index = df2.index + pd.DateOffset(months = 1)
df2

In [0]:
##Standard drop
decoded_metar.drop(decoded_metar.tail(72).index,inplace=True)
decoded_metar

##Manual drop
#decoded_metar.drop(decoded_metar.tail(14).index,inplace=True)
#decoded_metar

In [0]:
decoded_metar = decoded_metar.append(df2)
decoded_metar

In [0]:
##Standard check
decoded_metar.iloc[[-72]]

##Manual check
#decoded_metar.iloc[[-14]]

In [0]:
#Run once to get decoded_matar_master dataframe started
decoded_metar_master = decoded_metar

In [0]:
decoded_metar_master = decoded_metar_master.append(decoded_metar)

In [0]:
decoded_metar_master

In [0]:
#decoded_metar_master.to_csv('drive/My Drive/decoded_metar_master_through05.csv')
decoded_metar_master.to_csv('drive/My Drive/decoded_metar_master.csv')

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
###For when you mess up
#decoded_metar_master_through05 = pd.read_csv('drive/My Drive/decoded_metar_master_through05.csv')
#decoded_metar_master_through05 = decoded_metar_master_through05.set_index(keys='date_time')
#decoded_metar_master_through05
#decoded_metar_master = decoded_metar_master_through05

In [0]:
# Remove non-precipitation current weather by replacing with nan values and only keeping the non-nan values
decoded_metar_master.replace('HZ', np.nan, inplace=True)
decoded_metar_master.replace('MIFG', np.nan, inplace=True)
decoded_metar_master.replace('BCFG', np.nan, inplace=True)
decoded_metar_master.replace('PRFG', np.nan, inplace=True)
decoded_metar_master.replace('FG', np.nan, inplace=True)
decoded_metar_master.replace('FU', np.nan, inplace=True)
decoded_metar_master.replace('BR', np.nan, inplace=True)
decoded_metar_master.replace('FRFG', np.nan, inplace=True)
decoded_metar_master.replace('BLSN', np.nan, inplace=True)
decoded_metar_precip_only = decoded_metar_master[decoded_metar_master['current_wx1'].notna()]
decoded_metar_precip_only

In [0]:
# Create binary precip type data


# Create definition to apply labels of 1 for frozen precipitation and 0 for liquid precipitation
def label_precip (row):
    if row['current_wx1'] == '-RA' :
        return '0'
    if row['current_wx1'] == 'RA' :
        return '0'
    if row['current_wx1'] == '+RA' :
        return '0'
    if row['current_wx1'] == 'DZ' :
        return '0'
    if row['current_wx1'] == '-RA' :
        return '0'
    if row['current_wx1'] == '-TSRA' :
        return '0'
    if row['current_wx1'] == 'TSRA' :
        return '0'
    if row['current_wx1'] == '+TSRA' :
        return '0'
    if row['current_wx1'] == 'FZDZ' :
        return '1'
    if row['current_wx1'] == 'FZRA' :
        return '1'
    if row['current_wx1'] == '-SN' :
        return '1'
    if row['current_wx1'] == 'SN' :
        return '1'
    if row['current_wx1'] == '+SN' :
        return '1'
    if row['current_wx1'] == '-TSSN' :
        return '1'
    if row['current_wx1'] == 'TSSN' :
        return '1'
    if row['current_wx1'] == '+TSSN' :
        return '1'
    if row['current_wx1'] == '-SG' :
        return '1'
    if row['current_wx1'] == 'SG' :
        return '1'
    if row['current_wx1'] == '+SG' :
        return '1'
    if row['current_wx1'] == 'IC' :
        return '1'
    if row['current_wx1'] == '-PL' :
        return '1'
    if row['current_wx1'] == 'PL' :
        return '1'
    if row['current_wx1'] == '+PL' :
        return '1'
    else:
        return 'other'

# Apply the binary process to the asos dataframe
#decoded_metar_precip_only.apply (lambda row: label_precip(row), axis=1)

# Add the binary data column to the master dataframe
decoded_metar_precip_only['precip_type'] = decoded_metar_precip_only.apply (lambda row: label_precip(row), axis=1)
decoded_metar_precip_only


In [0]:
from sklearn.model_selection import train_test_split

In [0]:
# Separate into predictor and target variables
predictor_cols = ['wind_direction', 'wind_speed', 'current_wx1', 'low_cloud_type', 'low_cloud_level', 'medium_cloud_type', 'medium_cloud_level', 'high_cloud_type', 'high_cloud_level', 'highest_cloud_type', 'highest_cloud_level', 'cloud_coverage', 'air_temperature', 'dew_point_temperature', 'altimeter', 'present_weather', 'past_weather', 'air_pressure_at_sea_level', 'eastward_wind', 'northward_wind']
X = decoded_metar_precip_only[predictor_cols]
Y = decoded_metar_precip_only.precip_type
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
# Initiate the model using the default parameters
logreg = LogisticRegression()

# Fit the model
logreg.fit(X_train,Y_train)

# Predictions
ptype_predict=logreg.predict(X_test)

In [0]:
# Check skill with confusion matrix
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

In [0]:
# Plot confusion matrix

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')